# Imports

In [1]:
# import
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.actions.wheel_input import ScrollOrigin
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotInteractableException

from bs4 import BeautifulSoup
import requests
import pandas as pd
import sqlite3

from time import sleep
import random
import re

# Scraping Agora

In [2]:
# open browser
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get("https://agorahobby.com/store/mtg")

# obtain list of urls
urls = []
elements = driver.find_elements(By.CLASS_NAME, "sn_lvl1navi")
for element in elements:
    href = element.find_element(By.CLASS_NAME, "sn_toplink").get_attribute("href")
    
    if not href.endswith("#"):
        urls.append(href)
    else:
        inner_elements = element.find_elements(By.CLASS_NAME, "sn_link")
        for inner_element in inner_elements:
            urls.append((inner_element.get_attribute("href")))
            
# quit driver
driver.quit()

In [3]:
# create empty list
agora = []

In [ ]:
# loop through urls
for url in urls:
    # open browser and wait 2s
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
    driver.get(url)
    sleep(5)

    while True:
        try:
        # click on loadmore
            loadmore = driver.find_element(By.ID, "list_loadmore")
            loadmore.click()
            sleep(2)
        # once it hit ends of the page
        except Exception as e:
            print(f"Error clicking 'loadmore': {e}")
            break

    item_names = driver.find_elements(By.CLASS_NAME, "store-item-title")
    item_prices = driver.find_elements(By.CLASS_NAME, "store-item-price")
    for name, price in zip(item_names, item_prices):
        temp_dict = {}
        temp_dict['name'] = name.text
        temp_dict['price'] = float(price.text.replace('$', ''))
        agora.append(temp_dict)
    print(f"{url} is done!")
    print(agora[-1])
    
    driver.quit()

In [ ]:
agora_df = pd.DataFrame(agora)
agora_df['store'] = 'agora'
agora_df.to_csv('agora.csv')

# Scraping OneMTG

In [2]:
# Indicate base url
url = 'https://onemtg.com.sg/collections/mtg-singles-instock'
base_url = 'https://onemtg.com.sg/collections/mtg-singles-instock?page='
response = requests.get(url)
response.status_code

200

In [3]:
soup = BeautifulSoup(response.text, 'lxml')
num_pages = int(soup.find_all('div', class_='pages')[-1].text)
print('Number of pages (OneMTG): ', num_pages)

Number of pages (OneMTG):  3724


In [4]:
# create empty list
onemtg = []

In [5]:
# scraping
user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]

for i in range(num_pages):
    headers = {'User-Agent': random.choice(user_agent_list)}
    response = requests.get(base_url + str(i+1), headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', class_='product Norm')
    
    for item in items:
        temp_dict = {}
        title = item.find_all('p', class_='productTitle')[0].text.replace('\r\n', '').strip()
        price = item.find_all('p', class_='productPrice')[0].text.replace('\r\n', '').replace('$', '').replace(',', '').strip()
        if price == 'Varies':
            item_name = item.find_all('div', class_='addNow single')[0].text
            match = re.search(r'\$([\d.]+)', item_name)
            if match:
                price = match.group(1)
        temp_dict['name'] = title
        temp_dict['price'] = float(price)
        onemtg.append(temp_dict)
    
    sleep(random.uniform(2,5))
    if i % 100 == 0:
        print(onemtg[-1], response.status_code)

{'name': 'Big Score [Streets of New Capenna]', 'price': 1.7} 200
{'name': 'Farseek [Ravnica: City of Guilds]', 'price': 2.2} 200
{'name': 'Dusk Legion Zealot [Masters 25]', 'price': 0.4} 200
{'name': 'Volcanic Torrent [Commander Legends]', 'price': 0.5} 200
{'name': 'Ratadrabik of Urborg [Dominaria United]', 'price': 1.1} 200
{'name': 'Shimmer Myr [Kamigawa: Neon Dynasty Commander]', 'price': 0.5} 200
{'name': 'Primal Command [Strixhaven: School of Mages Mystical Archive]', 'price': 2.2} 200
{'name': 'Etali, Primal Storm (Timeshifted) [Time Spiral Remastered]', 'price': 2.8} 200
{'name': 'Fatal Grudge [Streets of New Capenna]', 'price': 0.5} 200
{'name': 'Bloodgift Demon [Commander 2014]', 'price': 2.4} 200
{'name': 'Insect Token [Dominaria Remastered Tokens]', 'price': 0.4} 200
{'name': 'Elvish Pioneer [Onslaught]', 'price': 0.36} 200
{'name': 'Student of Ojutai [Iconic Masters]', 'price': 0.4} 200
{'name': 'Elemental Token [Wilds of Eldraine Tokens]', 'price': 0.8} 200
{'name': 'Uurg

In [6]:
print("Number of cards (OneMTG): ", len(onemtg))

Number of cards (OneMTG):  44429


In [7]:
# create onemtg dataframe, convert price to float and create store column
onemtg_df = pd.DataFrame(onemtg)
onemtg_df['store'] = 'onemtg'

In [8]:
# save onemtg_df as csv
onemtg_df.to_csv("one_mtg.csv")

# Scraping CardsCitadel

In [38]:
# Indicate base url
url = 'https://cardscitadel.com/collections/mtg-singles-instock'
base_url = 'https://cardscitadel.com/collections/mtg-singles-instock?page='
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36"
} 
response = requests.get(url, headers=headers)
response.status_code

200

In [39]:
# print number of pages
soup = BeautifulSoup(response.text, 'lxml')
num_pages = int(soup.find_all('div', class_='pages')[-1].text)
print('Number of pages (Citadel): ', num_pages)

Number of pages (Citadel):  1706


In [42]:
# create empty list
citadel = []

In [43]:
# scraping
user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]

for i in range(num_pages):
    headers = {'User-Agent': random.choice(user_agent_list)}
    response = requests.get(base_url + str(i+1), headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', class_='product Norm')
    
    for item in items:
        temp_dict = {}
        title = item.find_all('p', class_='productTitle')[0].text.replace('\r\n', '').strip()
        price = item.find_all('p', class_='productPrice')[0].text.replace('\r\n', '').replace('$', '').replace(',', '').strip()
        if price == 'Varies':
            item_name = item.find_all('div', class_='addNow single')[0].text
            match = re.search(r'\$([\d.]+)', item_name)
            if match:
                price = match.group(1)
        temp_dict['name'] = title
        temp_dict['price'] = float(price)
        citadel.append(temp_dict)
    
    sleep(random.uniform(2,5))
    if i % 50 == 0:
        print(citadel[-1], response.status_code)

{'name': 'Mercurial Spelldancer [Phyrexia: All Will Be One]', 'price': 0.9} 200
{'name': 'Crippling Fear [Kaldheim]', 'price': 0.7} 200
{'name': 'Slicer, Hired Muscle // Slicer, High-Speed Antagonist [Universes Beyond: Transformers]', 'price': 2.8} 200
{'name': 'Daretti, Scrap Savant [Commander 2021]', 'price': 1.0} 200
{'name': 'Pithing Needle [Saviors of Kamigawa]', 'price': 7.0} 200
{'name': 'Reclusive Artificer [Magic Origins]', 'price': 0.3} 200
{'name': 'Braids, Arisen Nightmare [Game Day 2022]', 'price': 22.9} 200
{'name': 'Sheltered Thicket (Extended Art) [Doctor Who]', 'price': 1.0} 200
{'name': 'Baffling End [Rivals of Ixalan]', 'price': 0.4} 200
{'name': 'Spined Sliver [Time Spiral Timeshifted]', 'price': 3.8} 200
{'name': 'Selhoff Entomber [Innistrad: Crimson Vow]', 'price': 0.2} 200
{'name': 'Odric, Blood-Cursed [Innistrad: Double Feature]', 'price': 0.7} 200
{'name': 'Spectral Adversary [Innistrad: Midnight Hunt Prerelease Promos]', 'price': 2.0} 200
{'name': 'Ichor Shade

In [44]:
print("Number of cards (CardsCitadel): ", len(citadel))

Number of cards (CardsCitadel):  20461


In [45]:
# cleaning dataframe
citadel_df = pd.DataFrame(citadel)
citadel_df['store'] = 'citadel'

In [46]:
citadel_df.to_csv('citadel.csv')

# Scraping GreyOgreGames

In [27]:
# Indicate base url
url = 'https://www.greyogregames.com/search?page=1&q=**'
base_url = 'https://www.greyogregames.com/search?page={}&q=**'
response = requests.get(url)
response.status_code

200

In [28]:
# print number of pages
soup = BeautifulSoup(response.text, 'lxml')
page_list = []
for anchor in soup.find_all('ol', class_='pagination')[0].find_all('a'):
    if match := re.search(r'(\d+)', anchor.text):
            page_list.append(int(match.group(1)))
num_pages = max(page_list)
print('Number of pages (Grey Ogre Games): ', num_pages)

Number of pages (Grey Ogre Games):  5294


In [32]:
# create empty list
ogre = []

In [33]:
# scraping
user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_1) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
]

for i in range(num_pages):
    headers = {'User-Agent': random.choice(user_agent_list)}
    response = requests.get(base_url.format(i+1), headers=headers)
    soup = BeautifulSoup(response.text, 'lxml')
    items = soup.find_all('div', class_='productCard__lower')
    
    for item in items:
        name = item.find('p', class_='productCard__title').text.strip()
        price = item.find('p', class_='productCard__price').text.replace('$', '').replace('SGD', '').replace(',','').strip()
        try:
            price = float(price)
        except ValueError as e:
            print(e)
        ogre.append({
            'name': name,
            'price':price,
        })
    
    sleep(random.uniform(1,2))
    if i % 100 == 0:
        print(ogre[-1], response.status_code)

{'name': 'Plains (250)', 'price': 1.5} 200
{'name': 'Satoru Umezawa (Buy-A-Box)', 'price': 2.5} 200
{'name': 'Sram, Senior Edificer (Foil Etched)', 'price': 3.0} 200
{'name': 'Vexing Devil', 'price': 4.5} 200
{'name': "Master's Guide-Mural // Master's Manufactory", 'price': 0.5} 200
{'name': 'Crypt Incursion', 'price': 0.5} 200
{'name': 'Valduk, Keeper of the Flame (Retro)', 'price': 0.5} 200
{'name': 'Everythingamajig (Counter) (Unfinity Foil Edition)', 'price': 1.5} 200
{'name': 'Cartel Aristocrat', 'price': 0.5} 200
{'name': 'Huatli and Raptor in Bilbao - Pro Tour Rivals of Ixalan 2018', 'price': 80.0} 200
{'name': 'Falkenrath Perforator', 'price': 0.5} 200
{'name': 'Healing Technique (Extended Art)', 'price': 1.0} 200
{'name': 'Charix, the Raging Isle (386)', 'price': 1.0} 200
{'name': "Garruk's Uprising (Showcase)", 'price': 1.5} 200
{'name': 'Kin-Tree Warden', 'price': 0.5} 200
{'name': 'Drudge Skeletons', 'price': 9.5} 200
{'name': 'Geth, Lord of the Vault', 'price': 1.5} 200
{'

In [35]:
print("Number of cards (Ogre): ", len(ogre))

Number of cards (Ogre):  84703


In [36]:
# cleaning dataframe
ogre_df = pd.DataFrame(ogre)
ogre_df['store'] = 'greyogre'

In [48]:
ogre_df = ogre_df.drop(ogre_df[ogre_df['price'] == ''].index).reset_index(drop=True)

In [49]:
ogre_df['price'] = ogre_df['price'].astype(float)

In [53]:
ogre_df.to_csv('ogre.csv')

# Combining data and exporting it into SQLite3

In [2]:
ogre = pd.read_csv('ogre.csv', index_col=0)
citadel = pd.read_csv('citadel.csv', index_col=0)
agora = pd.read_csv('agora.csv', index_col=0)
onemtg = pd.read_csv('one_mtg.csv', index_col=0)

In [3]:
combined = pd.concat([agora, onemtg, citadel, ogre]).dropna().reset_index(drop=True)
combined = combined.drop(combined[combined['price'] == 0].index).reset_index(drop=True)
combined = combined.drop(combined[combined['name'].str.lower().str.contains('art card')].index)

In [13]:
# SQLite database connection
conn = sqlite3.connect('newcards.db')
cur = conn.cursor()

In [7]:
cur.execute("""CREATE TABLE cards (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    price REAL NOT NULL,
    store TEXT NOT NULL
    );""")

In [8]:
for _, row in combined.iterrows():
    conn.execute("INSERT INTO cards (name, price, store) VALUES (?, ?, ?)", (row[0], row[1], row[2]))

In [9]:
rows = cur.execute("SELECT * FROM cards;").fetchall()

In [11]:
# Commit changes and close connection
conn.commit()
conn.close()